# Read NPPES Raw

In [1]:
# from sqlalchemy import create_engine
# import pandas as pd
# db_connection_str = 'mysql+pymysql://root:@localhost:3306/physicianinfo?charset=utf8'
# db_connection = create_engine(db_connection_str)
# sql = 'SELECT * FROM physicianinfo.NPPESPhysician where MDDO = 1;'
# df = pd.read_sql(sql, con=db_connection)
# df.to_pickle('Data/NPPES/NPPESMDDOPhysician.p')
# print(len(df))

import pandas as pd
df = pd.read_pickle('Data/NPPES/NPPESMDDOPhysician.p')
print(len(df))
df.head()

1141176


,id,NPI,EntityType,LastName,FirstName,MiddleName,Prefix,Suffix,Gender,Credential,...,LastUpdateDate,BusinessMailingAddress,PracticeLocationAddress,OfficialInfo,OtherNameInfo,AffiliationInfo,DeactivationInfo,TaxonomyInfo,TaxonomyGroupInfo,OtherIdentifierInfo
0,1,1679576722,1,WIEBE,DAVID,A,None,None,M,MD,...,2007-07-08,"{""First Line"": ""PO BOX 2168"", ""City"": ""KEARNEY...","{""First Line"": ""3500 CENTRAL AVE"", ""City"": ""KE...",None,None,None,None,"[{""Taxonomy Code"": ""207X00000X"", ""License Numb...",None,"[{""Identifier"": ""645540"", ""Identifier Type Cod..."
1,2,1588667638,1,PILCHER,WILLIAM,C,DR.,None,M,MD,...,2014-05-29,"{""First Line"": ""1824 KING STREET"", ""Second Lin...","{""First Line"": ""1824 KING STREET"", ""Second Lin...",None,None,None,None,"[{""Taxonomy Code"": ""207RC0000X"", ""License Numb...",None,None
2,3,1215930367,1,GRESSOT,LAURENT,None,DR.,None,M,MD,...,2014-11-25,"{""First Line"": ""17323 RED OAK DR"", ""City"": ""HO...","{""First Line"": ""17323 RED OAK DR"", ""City"": ""HO...",None,None,None,None,"[{""Taxonomy Code"": ""174400000X"", ""License Numb...",None,None
3,4,1932102084,1,ADUSUMILLI,RAVI,K,None,None,M,MD,...,2012-04-23,"{""First Line"": ""2940 N MCCORD RD"", ""City"": ""TO...","{""First Line"": ""2940 N MCCORD RD"", ""City"": ""TO...",None,None,None,None,"[{""Taxonomy Code"": ""207RC0000X"", ""License Numb...",None,None
4,6,1750384806,1,BISBEE,ROBERT,None,DR.,None,M,MD,...,2017-03-10,"{""First Line"": ""5219 CITY BANK PKWY STE 214"", ...","{""First Line"": ""808 JOLIET AVE UNIT 120"", ""Cit...",None,None,None,None,"[{""Taxonomy Code"": ""207R00000X"", ""License Numb...",None,None


# Zip Information

In [3]:
import numpy as np
import json

def get_zip_code(x):
    try: 
        zipcode = int(json.loads(x)["Postal Code"]) 
    except: 
        zipcode = 0
    return zipcode

df['BusinessZip'] = df['BusinessMailingAddress'].apply(lambda x: get_zip_code(x))# .astype(int)
df['PracticeZip'] = df['PracticeLocationAddress'].apply(lambda x: get_zip_code(x))# .astype(int)
df['BusinessZip5'] = df['BusinessZip'].apply(lambda x: int(str(x)[:5]) if len(str(x)) in [5, 9] else np.nan)
df['PracticeZip5'] = df['PracticeZip'].apply(lambda x: int(str(x)[:5]) if len(str(x)) in [5, 9] else np.nan)

df.columns

Index(['id', 'NPI', 'EntityType', 'LastName', 'FirstName', 'MiddleName',
       'Prefix', 'Suffix', 'Gender', 'Credential', 'MDDO', 'Sole',
       'MainTaxonomy', 'CertificationDate', 'EnumerationDate',
       'LastUpdateDate', 'BusinessMailingAddress', 'PracticeLocationAddress',
       'OfficialInfo', 'OtherNameInfo', 'AffiliationInfo', 'DeactivationInfo',
       'TaxonomyInfo', 'TaxonomyGroupInfo', 'OtherIdentifierInfo',
       'BusinessZip', 'PracticeZip', 'BusinessZip5', 'PracticeZip5'],
      dtype='object')

# Specialty Information

In [6]:
path = 'Data/NPPES/nucc_taxonomy_211.csv'
RawData = pd.read_csv(path)

code2specialty = RawData[['Code', 'Display Name']]
code2specialty_dict = dict(zip(code2specialty['Code'].values, code2specialty['Display Name'].values))
df['Specialty'] = df['MainTaxonomy'].apply(lambda x: code2specialty_dict.get(x, None))

# Graduation Year

In [10]:
path = 'Data/DAC_NationalDownloadableFile.csv'

RawData = pd.read_csv(path, encoding='ISO-8859-1', low_memory=False)
RawData.columns = [i.strip() for i in RawData.columns]
RawData = RawData[- RawData['NPI'].duplicated()].reset_index(drop = True)
tmp = pd.merge(df[['NPI', 'LastName', 'FirstName', 'Gender', 'Credential', 'Sole', 
                   'BusinessZip', 'PracticeZip', 'Specialty', 'BusinessZip5', 'PracticeZip5']], 
               RawData[['NPI', 'Med_sch', 'Grd_yr', 'pri_spec', 'zip']],  on = 'NPI', how = 'left')
tmp

,NPI,LastName,FirstName,Gender,Credential,Sole,BusinessZip,PracticeZip,Specialty,BusinessZip5,PracticeZip5,Med_sch,Grd_yr,pri_spec,zip
0,1679576722,WIEBE,DAVID,M,MD,X,688482168,688472944,Orthopaedic Surgery Physician,68848.0,68847.0,NaN,NaN,NaN,NaN
1,1588667638,PILCHER,WILLIAM,M,MD,N,322044736,322044736,Cardiovascular Disease Physician,32204.0,32204.0,MEDICAL COLLEGE OF GEORGIA SCHOOL OF MEDICINE,1986.0,CARDIOVASCULAR DISEASE (CARDIOLOGY),320735572
2,1215930367,GRESSOT,LAURENT,M,MD,N,770901243,770901243,Specialist,77090.0,77090.0,NaN,NaN,NaN,NaN
3,1932102084,ADUSUMILLI,RAVI,M,MD,N,436151753,436151753,Cardiovascular Disease Physician,43615.0,43615.0,OTHER,1983.0,CARDIOVASCULAR DISEASE (CARDIOLOGY),436164922
4,1750384806,BISBEE,ROBERT,M,MD,N,794073537,794151148,Internal Medicine Physician,79407.0,79415.0,TEXAS TECH UNIVERSITY HEALTH SCIENCE CENTER SC...,1994.0,INTERNAL MEDICINE,794309406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141171,1669146106,BALOCH,ALI SHAHBAZ,M,MD,Y,606131744,606405759,Internal Medicine Physician,60613.0,60640.0,NaN,NaN,NaN,NaN
1141172,1447924881,IJAZ,ZAINAB,F,MD,N,480333849,480333849,Psychiatry Physician,48033.0,48033.0,NaN,NaN,NaN,NaN
1141173,1699449132,ROCA ALVAREZ,YULITH,F,MD,N,453423600,453423600,None,45342.0,45342.0,NaN,NaN,NaN,NaN
1141174,1083388425,AL OTOOM,OMAYMAH,F,MD,N,441091900,441091900,Psychiatry Physician,44109.0,44109.0,NaN,NaN,NaN,NaN


In [11]:
tmp[['NPI', 'LastName', 'FirstName']].to_csv('Output/NPI2Name.csv', index = False)

# Race

In [13]:
path = 'Data/NPPES/NPI2NameMDDORace.csv'
cols = ['NPI', 'race', 'asian', 'hispanic', 'nh_black', 'nh_white']
NPI2NameRace = pd.read_csv(path)[cols]
FinalData = pd.merge(tmp, NPI2NameRace, on = 'NPI', how = 'left')
FinalData

,NPI,LastName,FirstName,Gender,Credential,Sole,BusinessZip,PracticeZip,Specialty,BusinessZip5,PracticeZip5,Med_sch,Grd_yr,pri_spec,zip,race,asian,hispanic,nh_black,nh_white
0,1679576722,WIEBE,DAVID,M,MD,X,688482168,688472944,Orthopaedic Surgery Physician,68848.0,68847.0,NaN,NaN,NaN,NaN,nh_white,0.002915,0.007821,0.025477,0.963787
1,1588667638,PILCHER,WILLIAM,M,MD,N,322044736,322044736,Cardiovascular Disease Physician,32204.0,32204.0,MEDICAL COLLEGE OF GEORGIA SCHOOL OF MEDICINE,1986.0,CARDIOVASCULAR DISEASE (CARDIOLOGY),320735572,nh_white,0.001241,0.004728,0.017307,0.976723
2,1215930367,GRESSOT,LAURENT,M,MD,N,770901243,770901243,Specialist,77090.0,77090.0,NaN,NaN,NaN,NaN,nh_white,0.003474,0.015055,0.025027,0.956445
3,1932102084,ADUSUMILLI,RAVI,M,MD,N,436151753,436151753,Cardiovascular Disease Physician,43615.0,43615.0,OTHER,1983.0,CARDIOVASCULAR DISEASE (CARDIOLOGY),436164922,nh_white,0.184988,0.075427,0.020425,0.719160
4,1750384806,BISBEE,ROBERT,M,MD,N,794073537,794151148,Internal Medicine Physician,79407.0,79415.0,TEXAS TECH UNIVERSITY HEALTH SCIENCE CENTER SC...,1994.0,INTERNAL MEDICINE,794309406,nh_white,0.001802,0.004318,0.029559,0.964320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141171,1669146106,BALOCH,ALI SHAHBAZ,M,MD,Y,606131744,606405759,Internal Medicine Physician,60613.0,60640.0,NaN,NaN,NaN,NaN,nh_white,0.220066,0.106605,0.107429,0.565900
1141172,1447924881,IJAZ,ZAINAB,F,MD,N,480333849,480333849,Psychiatry Physician,48033.0,48033.0,NaN,NaN,NaN,NaN,nh_white,0.286878,0.260878,0.034607,0.417637
1141173,1699449132,ROCA ALVAREZ,YULITH,F,MD,N,453423600,453423600,None,45342.0,45342.0,NaN,NaN,NaN,NaN,hispanic,0.000366,0.983420,0.000612,0.015601
1141174,1083388425,AL OTOOM,OMAYMAH,F,MD,N,441091900,441091900,Psychiatry Physician,44109.0,44109.0,NaN,NaN,NaN,NaN,asian,0.333297,0.123595,0.247699,0.295409


# Zip Information

In [16]:
zipSeries = pd.Series(list(set(FinalData['BusinessZip'].to_list() + FinalData['PracticeZip'].to_list())))
zipSeries.apply(lambda x: len(str(x))).value_counts().sort_index()

1          7
2         11
3        247
4       1550
5      13152
6         85
7       8197
8      33526
9     322936
10         1
dtype: int64

In [17]:
zipcode2freq = zipSeries.apply(lambda x: str(x)[:5] if len(str(x)) in [5, 9] else np.nan).value_counts().sort_index()
zipcode2freq

10001    107
10002     74
10003    258
10004     16
10005     21
        ... 
99925      2
99926      1
99928      1
99929      3
99999      1
Length: 20687, dtype: int64

In [ ]:
import pandas as pd
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)

L = []
for zipcode in zipcode2freq.index:
    zipcode = str(zipcode)
    # print(zipcode)
    
    d = {}
    try:
        info = search.by_zipcode(zipcode).to_dict()
        d['zipcode'] = zipcode
        d['population'] = info['population']
        d['population_density'] = info['population_density']
        L.append(d)
    except:
        print(zipcode)

Zip2Population = pd.DataFrame(L)
Zip2Population.to_pickle('Output/Zip2Population.p')
Zip2Population